In [1]:
from pathlib import Path
import shutil

def delete_phy_folders(root_path: str, max_depth: int = 7, dry_run: bool = True) -> None:
    root = Path(root_path)
    phy_folders = []
    
    def find_phy_folders(current_path: Path, current_depth: int) -> None:
        if current_depth > max_depth:
            return
        
        for item in current_path.iterdir():
            if item.is_dir():
                if item.name == ".phy":
                    phy_folders.append(item)
                else:
                    find_phy_folders(item, current_depth + 1)
    
    find_phy_folders(root, 0)
    
    if not phy_folders:
        print("No .phy folders found")
        return
    
    print(f"Found {len(phy_folders)} .phy folder(s):")
    for folder in phy_folders:
        print(f"  {folder}")
    
    if dry_run:
        print("\nDry run mode - no folders deleted")
        print("Set dry_run=False to actually delete")
        return
    
    for folder in phy_folders:
        shutil.rmtree(folder)
        print(f"Deleted: {folder}")

if __name__ == "__main__":
    path = r"F:\AK_data\rawdata"
    delete_phy_folders(path, max_depth=7, dry_run=False)
    # delete_phy_folders(path, max_depth=7, dry_run=False)

Found 147 .phy folder(s):
  F:\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250304_01\ephys\Ivy_250304_162606\kilosort4_old\.phy
  F:\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250304_01\ephys\Ivy_250304_163709_skip\kilosort4\.phy
  F:\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250306_01\ephys\Ivy_250306_112647\kilosort4\.phy
  F:\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250306_01\ephys\Ivy_250306_112647\kilosort4_old2\.phy
  F:\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250306_01\ephys\Ivy_250306_112647\kilosort4_sorted\.phy
  F:\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250306_01\ephys\Ivy_250306_112647\kilosort_old\.phy
  F:\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250307_01\ephys\Ivy_250307_110513\kilosort4\.phy
  F:\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250307_01\ephys\Ivy_250307_110513\kilosort4test24\.phy
  F:\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250307_01\ephys\Ivy_250307_110513\kilosort4test26\.phy
  F:\AK_data\rawdata\sub-01_id-Ivy\ses-000_date

### Code

In [ ]:
df

In [ ]:
%load_ext autoreload
%autoreload 2

from movement.io import load_poses
from movement.filtering import rolling_filter
from movement.kinematics import compute_pairwise_distances, compute_velocity, compute_acceleration, compute_forward_vector_angle, compute_time_derivative
from movement.utils.vector import compute_norm, cart2pol

from ethograph.utils.io import TrialTree, add_angle_rgb_to_ds, get_session_path, add_changepoints_to_ds, extract_trial_info_from_filename, set_media_attrs
from ethograph.features.mov_features import compute_distance_to_constant, Position3DCalibration
from ethograph.features.changepoints import find_troughs_binary, find_nearest_turning_points_binary
from ethograph.features.preprocessing import gaussian_smoothing
from scipy.io import loadmat
import glob
import os
import json
import warnings
import numpy as np
from pathlib import Path

warnings.filterwarnings(
    'ignore', 
    message='Confidence array was not provided.Setting to an array of NaNs',
    module='movement.validators.datasets'
)

user = "Alice"
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
json_file = os.path.join(desktop_path, "user_paths.json")
with open(json_file, "r") as file:
    paths = json.load(file)
raw_videos_folder = paths[user]["raw_videos_folder"]



fps = 200
good_s3d_feats = [ 326, 327, 292, 363, 219, 192, 260, 66, 332, 199,
       288, 763, 837, 182, 24, 218, 213, 21, 733, 242]

disp_xyz = [-10.23, -5.907, -1.395]
box1 = np.array([-7.08514973,  0.14055037,  0.58930513]) # front left
box2 = np.array([-6.97786264,  9.87752058,  0.96104736]) # back left
box3 = np.array([6.81007923, 9.72017014, 0.77115876]) # back right
box4 = np.array([ 6.77518696, -0.02424044,  0.60184316]) # front right


calibration = Position3DCalibration()
smoothing_params = {"sigma": 1.5, "axis": 0, "mode": "constant", "cval": np.nan}
clip_distance = 50 # cm (no distance feature exceeds this)

# raw!
behav_folders = [
                #  r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250527_01\behav",
                #  r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250527_02\behav",
                #  r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250528_01\behav",            
                #  r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250526_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250526_02\behav",
                # r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250528_02\behav",
                # r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250529_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250530_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250602_01\behav",
                
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250306_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250307_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250308_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250309_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250503_02\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250504_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250505_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250506_02\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250507_02\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250507_03\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250508_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250508_02\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250509_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250512_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250513_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250514_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250515_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250516_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250519_01\behav",
                # r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250521_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250522_01\behav"       
                ]



for behav_folder in behav_folders:
    print(f"Processing folder: {behav_folder}")

    dlc_3d_paths = np.sort(glob.glob(os.path.join(behav_folder, "dlc", "*_3D.csv")))


    ds_list = []
    for dlc_3d_path in dlc_3d_paths:
        ds = load_poses.from_dlc_file(dlc_3d_path, fps=fps)
        
        session_date, trial_num, bird = extract_trial_info_from_filename(dlc_3d_path)
        ds = ds.assign_coords(individuals=[bird])
        ds.attrs["trial"] = trial_num # can be number or string, has to be unique per session
        ds.attrs["session"] = f"{session_date}_{str(Path(behav_folder).parent)[-2:]}"  # e.g. "20250527_sub-03_id-Freddy"
        ds.attrs["bird"] = bird


        base_name = os.path.basename(dlc_3d_path)
        dlc_suffix = "DLC_resnet50_Felix_cross_SessionsAug1shuffle1_200000_filtered"
        
        ds = set_media_attrs(
            ds,
            cameras=[
                base_name.replace("_DLC_3D.csv", "-cam-1.mp4"),
                base_name.replace("_DLC_3D.csv", "-cam-2.mp4"),
            ],
            tracking=[
                base_name.replace("_DLC_3D.csv", f"-cam-1{dlc_suffix}.csv"),
                base_name.replace("_DLC_3D.csv", f"-cam-2{dlc_suffix}.csv"),
            ],
            tracking_prefix="dlc",
        )
        

   
        ds = calibration.transform(ds) 
        ds["position"] = gaussian_smoothing(ds.position, **smoothing_params)
        

    



        
        ds["velocity"] = compute_velocity(ds.position.sel(keypoints=['stickTip', 'beakTip'])).clip(min=-150, max=150)
        ds["speed"] = compute_norm(ds.velocity.sel(keypoints=['stickTip', 'beakTip']))
        
        
        smoothing_params_doubled = smoothing_params.copy()
        smoothing_params_doubled["sigma"] *= 2
        position_very_smooth = gaussian_smoothing(ds.position, **smoothing_params_doubled)
        position_very_smooth = gaussian_smoothing(ds.position, **smoothing_params_doubled)
        ds["acceleration"] = compute_acceleration(position_very_smooth.sel(keypoints=['stickTip', 'beakTip'])).clip(min=-1500, max=1500)
        
        

        # Distances
        ds["pellet_beakTip_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"pellet": "beakTip"}).clip(0, clip_distance)
        ds["pellet_stickTip_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"pellet": "stickTip"}).clip(0, clip_distance)
        ds["disp_beakTip_dist"] = compute_distance_to_constant(ds.position, reference_point=disp_xyz, keypoint="beakTip").clip(0, clip_distance)
        ds["disp_stickTip_dist"] = compute_distance_to_constant(ds.position, reference_point=disp_xyz, keypoint="stickTip").clip(0, clip_distance)
        
        ds["sticktip_cornerLFront_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"stickTip": "box1"}).clip(0, clip_distance)
        ds["sticktip_cornerLBack_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"stickTip": "box2"}).clip(0, clip_distance)
        ds["sticktip_cornerRBack_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"stickTip": "box3"}).clip(0, clip_distance)
        ds["sticktip_cornerRFront_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"stickTip": "box4"}).clip(0, clip_distance)

        ds["beakTip_cornerLFront_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"beakTip": "box1"}).clip(0, clip_distance)
        ds["beakTip_cornerLBack_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"beakTip": "box2"}).clip(0, clip_distance)
        ds["beakTip_cornerRBack_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"beakTip": "box3"}).clip(0, clip_distance)
        ds["beakTip_cornerRFront_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"beakTip": "box4"}).clip(0, clip_distance)

        

        
        # Only keep subset of keypoints
        ds = ds.sel(keypoints=["beakTip", "stickTip", "pellet"])


        s3d_file = base_name.replace("_DLC_3D.csv", "-cam-1_s3d.npy")
        s3d_path = os.path.join(behav_folder, "s3d", s3d_file)
        s3d_data = np.load(s3d_path)
        ds["s3d"] = (("time", "s3d_dims"), s3d_data[:, good_s3d_feats])


        for var in ds.data_vars:
            ds[var].attrs["type"] = "features"
            
        
        ds = add_changepoints_to_ds(
            ds=ds, 
            target_feature="speed",
            changepoint_name="troughs", 
            changepoint_func=find_troughs_binary, 
            prominence=0.5, 
            distance=2
        )
        
        ds = add_changepoints_to_ds(
            ds=ds,
            target_feature="speed",
            changepoint_name="turning_points",
            changepoint_func=find_nearest_turning_points_binary,
            threshold=1.0,
            max_value=50,
            prominence=5,
            width=2
        )
        
        ds = add_angle_rgb_to_ds(ds, smoothing_params=smoothing_params)
        
        
        ds["labels"] = (("time", "individuals"), np.zeros((len(ds.coords["time"]), len(ds.coords["individuals"]))))
        
        
        
        # From matlab
        deriv_behav_folder = behav_folder.replace("rawdata", "derivatives")
        AllTrialsPath = os.path.join(deriv_behav_folder, "Trial_data.mat")
        
        mat_data = loadmat(AllTrialsPath, squeeze_me=True, struct_as_record=False)
        AllTrials = mat_data['AllTrials']
        
        for i, trial in enumerate(AllTrials):
            if trial.trial_num == trial_num:
                ds.attrs["poscat"] = trial.info.poscat
                ds.attrs["num_pellets"] = trial.info.num_pellets
              
                

                event_data = np.zeros(4)  
                
                disp_out_in = trial.info.stick_in_out_disp
                disp_out = int(disp_out_in[0]) -1
                
                for j, val in enumerate([0, 1]):
                    try:
                        event_data[j] = int(disp_out_in[val]) - 1
                    except Exception:
                        event_data[j] = np.nan
                for j, idx in enumerate([0, 1], start=2):
                    try:
                        event_data[j] = int(trial.info.first_in_last_out[idx-2]) - 1
                    except Exception:
                        event_data[j] = np.nan

                ds["boundary_events"] = ("events", event_data)
                ds.assign_coords(events=("events", ["disp_out", "disp_in", "box_in", "box_out"]))
    

        ds_list.append(ds)


    deriv_behav_folder = behav_folder.replace("rawdata", "derivatives")
    dt = TrialTree.from_datasets(ds_list)
    dt.to_netcdf(os.path.join(deriv_behav_folder, "Trial_data.nc"))

In [1]:
path = r"C:/Users/aksel/Documents/Code/EthoGraph/data/Night_of_the_Living_Dead/MovieString.nc"

from ethograph.utils.io import TrialTree
dt = TrialTree.load(path)
ds = dt.itrial(0)

In [ ]:
dt

In [2]:
dt

<xarray.DataTree>
Group: /
└── Group: /trial_movie1
        Dimensions:                  (time: 123690, individuals: 3)
        Coordinates:
          * time                     (time) float64 990kB 0.0 0.04167 ... 5.154e+03
          * individuals              (individuals) <U7 84B 'Ben' 'Barbara' 'Harry'
        Data variables:
            envelope                 (time) float64 990kB ...
            labels                   (time, individuals) float64 3MB ...
            envelope_troughs         (time) int8 124kB ...
            envelope_turning_points  (time) int8 124kB ...
        Attributes:
            cameras:  cam1
            cam1:     movie.mp4
            mics:     mic1
            mic1:     movie.wav
            fps:      24
            sr:       44100
            trial:    movie1

In [3]:
dt.trials

['movie1']